In [1]:
import numpy as np
import pandas as pd
import sklearn
import warnings

In [2]:
# let say i havemore than 20 or more columns
# i wan to see each and every column
# but if i do pd.columns it will show some startinhg and ending columns
# but if i want  to see whole columns
pd.set_option('display.max_columns', None)

In [3]:

sklearn.set_config(transform_output="pandas")
#when we use and transformer like fucntion , or power transformer
# or anything  it will return numpy array
#if we want a output in pandas df so we will use by changing the sklearn config


In [4]:
warnings.filterwarnings('ignore')

In [5]:
file_path="C:\\Users\\harsh\\OneDrive\\Desktop\\Flight_price_prediction\\Data\\train.csv"
train=pd.read_csv(file_path)
train.head()

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-04-01,Kolkata,Banglore,21:00:00,02:15:00,5h 15m,1.0,No Info,5930
1,Indigo,2019-03-09,Chennai,Kolkata,05:15:00,07:35:00,2h 20m,0.0,No Info,6482
2,Air India,2019-04-09,Delhi,Cochin,20:15:00,19:15:00,23h,2.0,No Info,9916
3,Jet Airways,2019-06-01,Delhi,Cochin,18:15:00,19:00:00,24h 45m,1.0,No Info,14714
4,Multiple Carriers,2019-05-27,Delhi,Cochin,06:00:00,12:35:00,6h 35m,1.0,No Info,11264


In [6]:
X_train= train.drop(columns='price')
y_train=train['price']


# Transformation  on every column

In [7]:
X_train.columns.tolist()

['airline',
 'date_of_journey',
 'source',
 'destination',
 'dep_time',
 'arrival_time',
 'duration',
 'total_stops',
 'additional_info']

In [8]:
X_train.airline

0                 Indigo
1                 Indigo
2              Air India
3            Jet Airways
4      Multiple Carriers
             ...        
635          Jet Airways
636            Air India
637               Indigo
638                Goair
639                Goair
Name: airline, Length: 640, dtype: object

In [18]:
# 1 IMPUTATION
# see in eda most of the percentage is coverd by Jetairways, indigo airindia , multiplecarriers
# rest 4 are lest then 15 percent so we will make them other
# and then we will apply ohe
from sklearn.impute import SimpleImputer
from feature_engine.encoding import RareLabelEncoder, MeanEncoder
# with rare label encoder we will group categories jii  percent m hme krna hai
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder,
    MinMaxScaler,
    PowerTransformer,
    FunctionTransformer
)

In [10]:
air_transformer=Pipeline(steps=[
    ("imputer", SimpleImputer(strategy='most_frequent')),
    ('grouper', RareLabelEncoder(tol=0.1, replace_with='Other', n_categories=2)),
    # see tol=0.1 vo categoreis lo jinki percent isless that 10
    # replace_with='others namm other krdo
    # n_categores=2 ye ta krna jab atleast ase 2 categories ho tab 
    # hmare pass 4 hai to hojyga
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))

    


])

air_transformer.fit(X_train.loc[:,['airline']])


Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('grouper',
                 RareLabelEncoder(n_categories=2, replace_with='Other',
                                  tol=0.1)),
                ('encoder',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

# Date Of Journey

In [11]:
X_train.date_of_journey
# see date time m object class hai to  we will solve this ina unique way
# i will extractv eachand everything day, date , month and everythng from featurengine
# and then we wll scale them bcoz  mahine ak din 25 ha
# or week will be 1,2 till 7 so scale ka issue na ho sliye scale krenge

0      2019-04-01
1      2019-03-09
2      2019-04-09
3      2019-06-01
4      2019-05-27
          ...    
635    2019-04-01
636    2019-03-01
637    2019-05-27
638    2019-03-09
639    2019-05-15
Name: date_of_journey, Length: 640, dtype: object

In [12]:
from feature_engine.datetime import DatetimeFeatures

In [13]:
feature_to_extract=['month', 'week', 'day_of_week', 'day_of_year']
doj_transformer=Pipeline(steps=[
    ('dt', DatetimeFeatures(features_to_extract=feature_to_extract, yearfirst=True,format='mixed')),
    ('scaler', MinMaxScaler())
])

doj_transformer.fit_transform(X_train.loc[:,['date_of_journey'] ])

,date_of_journey_month,date_of_journey_week,date_of_journey_day_of_week,date_of_journey_day_of_year
0,0.333333,0.294118,0.000000,0.262712
1,0.000000,0.058824,0.833333,0.067797
2,0.333333,0.352941,0.166667,0.330508
3,1.000000,0.764706,0.833333,0.779661
4,0.666667,0.764706,0.000000,0.737288
...,...,...,...,...
635,0.333333,0.294118,0.000000,0.262712
636,0.000000,0.000000,0.666667,0.000000
637,0.666667,0.764706,0.000000,0.737288
638,0.000000,0.058824,0.833333,0.067797


In [21]:
#now we have column des and source jo same hi hai
# to we will handle them 
# Group rare labels
# means encoded
# power trasformer

location_subset=X_train.loc[:, ['source','destination']]
location_subset

,source,destination
0,Kolkata,Banglore
1,Chennai,Kolkata
2,Delhi,Cochin
3,Delhi,Cochin
4,Delhi,Cochin
...,...,...
635,Kolkata,Banglore
636,Chennai,Kolkata
637,Delhi,Cochin
638,Delhi,Cochin


In [22]:
# Mean encoded
# ye kya krta hai
# let say you have 5 categories in columne and your target isnumeric col like price
# so vo category ke lie uske avg price clalculate krke deta hai

location_pip1=Pipeline(steps=[
    ('grouper', RareLabelEncoder(tol=0.1,replace_with='Other', n_categories=2)),
    ('encoder', MeanEncoder()),
    ('scaler', PowerTransformer())
    # ye powertransform ky kr rha hai numeric colu hmare target column ki cymetrric kredega
    # kind of norml
])

location_pip1.fit_transform(location_subset, y_train)

,source,destination
0,-0.359737,-0.353698
1,-1.816042,-0.763473
2,1.068361,1.069016
3,1.068361,1.069016
4,1.068361,1.069016
...,...,...
635,-0.359737,-0.353698
636,-1.816042,-0.763473
637,1.068361,1.069016
638,1.068361,1.069016


In [ ]:
# Mean encoded
# ye kya krta hai
# let say you have 5 categories in columne and your target isnumeric col like price
# so vo category ke lie uske avg price clalculate krke deta hai



# see for soruce and destination i want to check if the city is from north or not i will make a temp coloum
# i will make a function but that function will not work in sklearn or pipeline
# so do that i will pass that function in Function Transformer
# we are doing this tolearn hw we can use FUnction transformer

In [23]:
np.union1d(
    X_train.source.unique(),
    X_train.destination.unique()
)
# see we have this unique  cities

array(['Banglore', 'Chennai', 'Cochin', 'Delhi', 'Hyderabad', 'Kolkata',
       'Mumbai', 'New Delhi'], dtype=object)

In [28]:
def is_north(X):
    columns=X.columns.to_list()
    north_cities=['Delhi', 'Kolkata', 'Mumbai', 'New Delhi']

    return(
        X
        .assign(**{
           f"{col}_is_north" : X.loc[:,col].isin(north_cities).astype(int)
            for col in columns
        })
        .drop(columns=columns)
    )

is_north(location_subset)
# se eye function will not gonna work in dsklearn to make 
# it work we will use function transformer

FunctionTransformer(func=is_north).fit_transform(location_subset)

,source_is_north,destination_is_north
0,1,0
1,0,1
2,1,0
3,1,0
4,1,0
...,...,...
635,1,0
636,0,1
637,1,0
638,1,0


In [29]:
# now how we will mix this func transformer with 
# the steps we did above for source and destination
# we did 3 steps power transformer, mean encoded, rare labelencoder
# for that we  gonna import Feature union
from sklearn.pipeline import FeatureUnion


In [30]:
location_transformer=FeatureUnion(transformer_list=[
    ('part1',location_pip1),
    ('part2', FunctionTransformer(func=is_north))
])

location_transformer.fit_transform(location_subset,y_train)
# y_train isliye kuki mean encoded bhi kr rhe
# see ase we transform differen thinghs for same variables

,source,destination,source_is_north,destination_is_north
0,-0.359737,-0.353698,1,0
1,-1.816042,-0.763473,0,1
2,1.068361,1.069016,1,0
3,1.068361,1.069016,1,0
4,1.068361,1.069016,1,0
...,...,...,...,...
635,-0.359737,-0.353698,1,0
636,-1.816042,-0.763473,0,1
637,1.068361,1.069016,1,0
638,1.068361,1.069016,1,0


# Dep time and Arrival Time

In [32]:
X_train['dep_time']
# we have hour minutes and second

0      21:00:00
1      05:15:00
2      20:15:00
3      18:15:00
4      06:00:00
         ...   
635    06:30:00
636    11:40:00
637    07:30:00
638    06:00:00
639    07:45:00
Name: dep_time, Length: 640, dtype: object

In [33]:
time_subset=X_train.loc[:, ['dep_time', 'arrival_time']]

In [34]:
time_subset

,dep_time,arrival_time
0,21:00:00,02:15:00
1,05:15:00,07:35:00
2,20:15:00,19:15:00
3,18:15:00,19:00:00
4,06:00:00,12:35:00
...,...,...
635,06:30:00,16:20:00
636,11:40:00,13:55:00
637,07:30:00,21:00:00
638,06:00:00,13:20:00


In [38]:
time_pipe1=Pipeline(steps=[
    ('dt',DatetimeFeatures(features_to_extract=['hour', 'minute']) ),
    # applying scalin
    ('scaler', MinMaxScaler())
])

time_pipe1.fit_transform(time_subset)

,dep_time_hour,dep_time_minute,arrival_time_hour,arrival_time_minute
0,0.913043,0.000000,0.086957,0.272727
1,0.217391,0.272727,0.304348,0.636364
2,0.869565,0.272727,0.826087,0.272727
3,0.782609,0.272727,0.826087,0.000000
4,0.260870,0.000000,0.521739,0.636364
...,...,...,...,...
635,0.260870,0.545455,0.695652,0.363636
636,0.478261,0.727273,0.565217,1.000000
637,0.304348,0.545455,0.913043,0.000000
638,0.260870,0.000000,0.565217,0.363636


# now againg we will do some custom transformation

In [42]:
def part_of_day(X, morning=4, noon=12, eve=16, night=20):
	columns = X.columns.to_list()
	X_temp = X.assign(**{
		col: pd.to_datetime(X.loc[:, col]).dt.hour
		for col in columns
	})

	return (
		X_temp
		.assign(**{
			f"{col}_part_of_day": np.select(
				[X_temp.loc[:, col].between(morning, noon, inclusive="left"),
				 X_temp.loc[:, col].between(noon, eve, inclusive="left"),
				 X_temp.loc[:, col].between(eve, night, inclusive="left")],
				["morning", "afternoon", "evening"],
				default="night"
			)
			for col in columns
		})
		.drop(columns=columns)
	)
#part_of_day(time_subset)

FunctionTransformer(func=part_of_day).fit_transform(time_subset)

,dep_time_part_of_day,arrival_time_part_of_day
0,night,night
1,morning,morning
2,night,evening
3,evening,evening
4,morning,afternoon
...,...,...
635,morning,evening
636,morning,afternoon
637,morning,night
638,morning,afternoon


In [43]:
# now we will do count encoding
# har category jitni baar ayi vo count ajyga
#like morning =20 bar aya to har morning ki jaga uska count ajyaga 
# and the we will do scaling
from feature_engine.encoding import CountFrequencyEncoder


In [45]:
time_pipe2=Pipeline(steps=[
    ('part', FunctionTransformer(func=part_of_day)),
    ('encoder', CountFrequencyEncoder()),
    ('scaler', MinMaxScaler())


])
time_pipe2.fit_transform(time_subset)

,dep_time_part_of_day,arrival_time_part_of_day
0,0.146018,1.000000
1,1.000000,0.877193
2,0.146018,0.508772
3,0.163717,0.508772
4,1.000000,0.000000
...,...,...
635,1.000000,0.508772
636,1.000000,0.000000
637,1.000000,1.000000
638,1.000000,0.000000


# now i have timepip1 in which i extract houra and minutes and scaled them

# in timepipe2  we extract part of the day then count their appearance and  scale them as well 
now will make them togther

In [47]:
time_transformer=FeatureUnion(transformer_list=[
    ('part1', time_pipe1),
    ('part2', time_pipe2)
])

time_transformer.fit_transform(time_subset)

,dep_time_hour,dep_time_minute,arrival_time_hour,arrival_time_minute,dep_time_part_of_day,arrival_time_part_of_day
0,0.913043,0.000000,0.086957,0.272727,0.146018,1.000000
1,0.217391,0.272727,0.304348,0.636364,1.000000,0.877193
2,0.869565,0.272727,0.826087,0.272727,0.146018,0.508772
3,0.782609,0.272727,0.826087,0.000000,0.163717,0.508772
4,0.260870,0.000000,0.521739,0.636364,1.000000,0.000000
...,...,...,...,...,...,...
635,0.260870,0.545455,0.695652,0.363636,1.000000,0.508772
636,0.478261,0.727273,0.565217,1.000000,1.000000,0.000000
637,0.304348,0.545455,0.913043,0.000000,1.000000,1.000000
638,0.260870,0.000000,0.565217,0.363636,1.000000,0.000000


In [16]:
# lets use column Transformer
#ye jo pipeline bnai vvo bnia mne sirf airline k lie bnayi har column k lie alag se bnegi right
# so for that we gonna use column transformer
from sklearn.compose import ColumnTransformer

,air__airline_Air India,air__airline_Indigo,air__airline_Jet Airways,air__airline_Multiple Carriers,air__airline_Other,doj__date_of_journey_month,doj__date_of_journey_week,doj__date_of_journey_day_of_week,doj__date_of_journey_day_of_year,location__source,location__destination,location__source_is_north,location__destination_is_north,time__dep_time_hour,time__dep_time_minute,time__arrival_time_hour,time__arrival_time_minute,time__dep_time_part_of_day,time__arrival_time_part_of_day,remainder__duration,remainder__total_stops,remainder__additional_info
0,0.0,1.0,0.0,0.0,0.0,0.333333,0.294118,0.000000,0.262712,-0.359737,-0.353698,1,0,0.913043,0.000000,0.086957,0.272727,0.146018,1.000000,5h 15m,1.0,No Info
1,0.0,1.0,0.0,0.0,0.0,0.000000,0.058824,0.833333,0.067797,-1.816042,-0.763473,0,1,0.217391,0.272727,0.304348,0.636364,1.000000,0.877193,2h 20m,0.0,No Info
2,1.0,0.0,0.0,0.0,0.0,0.333333,0.352941,0.166667,0.330508,1.068361,1.069016,1,0,0.869565,0.272727,0.826087,0.272727,0.146018,0.508772,23h,2.0,No Info
3,0.0,0.0,1.0,0.0,0.0,1.000000,0.764706,0.833333,0.779661,1.068361,1.069016,1,0,0.782609,0.272727,0.826087,0.000000,0.163717,0.508772,24h 45m,1.0,No Info
4,0.0,0.0,0.0,1.0,0.0,0.666667,0.764706,0.000000,0.737288,1.068361,1.069016,1,0,0.260870,0.000000,0.521739,0.636364,1.000000,0.000000,6h 35m,1.0,No Info
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,0.0,0.0,1.0,0.0,0.0,0.333333,0.294118,0.000000,0.262712,-0.359737,-0.353698,1,0,0.260870,0.545455,0.695652,0.363636,1.000000,0.508772,9h 50m,1.0,In-flight meal not included
636,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.666667,0.000000,-1.816042,-0.763473,0,1,0.478261,0.727273,0.565217,1.000000,1.000000,0.000000,2h 15m,0.0,No Info
637,0.0,1.0,0.0,0.0,0.0,0.666667,0.764706,0.000000,0.737288,1.068361,1.069016,1,0,0.304348,0.545455,0.913043,0.000000,1.000000,1.000000,13h 30m,1.0,No Info
638,0.0,0.0,0.0,0.0,1.0,0.000000,0.058824,0.833333,0.067797,1.068361,1.069016,1,0,0.260870,0.000000,0.565217,0.363636,1.000000,0.000000,7h 20m,1.0,No Info


# Duration

1. handle outliers
2. handle missing values
3. make 4 different types of transformation

In [ ]:
# def flight_time(X, short=180, medium=400, eve=16, night=20):
# 	columns = X.columns.to_list()
# 	X_temp = X.assign(**{
# 		col: pd.to_datetime(X.loc[:, col]).dt.hour
# 		for col in columns
# 	})

# 	return (
# 		X_temp
# 		.assign(**{
# 			f"{col}_part_of_day": np.select(
# 				[X_temp.loc[:, col].between(morning, noon, inclusive="left"),
# 				 X_temp.loc[:, col].between(noon, eve, inclusive="left"),
# 				 X_temp.loc[:, col].between(eve, night, inclusive="left")],
# 				["morning", "afternoon", "evening"],
# 				default="night"
# 			)
# 			for col in columns
# 		})
# 		.drop(columns=columns)
# 	)

# FunctionTransformer(func=part_of_day).fit_transform(time_subset)

In [55]:
dur_subset=X_train.loc[:,'duration']
dur_subset

0       5h 15m
1       2h 20m
2          23h
3      24h 45m
4       6h 35m
        ...   
635     9h 50m
636     2h 15m
637    13h 30m
638     7h 20m
639     2h 55m
Name: duration, Length: 640, dtype: object

In [ ]:
# we will do rvf kernel
# it help us in calculating sim between 2 values
# we will  calculate percentle of duration
# and then we will calculate of every value similarity with percentile
# we gonna use RBF Kernel

In [56]:
from sklearn.base import BaseEstimator , TransformerMixin

In [63]:
class RBFPercentileSimilarity(BaseEstimator, TransformerMixin):
    def __init__(self, variables=None, percentiles=[0.25,0.5,0.75], gamma=0.1):
        self.variables= variables
        # jo bhi varaibles denge user uspr RBF lagaynge 
        self.percentiles=percentiles
        self.gamma=gamma

    def fit(self,X, y=None):
        
    

In [64]:
RBFPercentileSimilarity

__main__.RBFPercentileSimilarity

# Coloum transformer

In [ ]:
column_transformer=ColumnTransformer(transformers=[
    ('air', air_transformer, ['airline']),
    ('doj', doj_transformer, ['date_of_journey']),
    ('location', location_transformer, ['source', 'destination']),
    ('time', time_transformer, ['dep_time', 'arrival_time'])
], remainder='passthrough')

column_transformer.fit_transform(X_train, y_train)